In [ ]:
import pandas as pd
import geopandas as gpd
import os

In [ ]:
# --- Rutas de Archivos (Configurables) ---
RUTA_CENSO_CSV_DIR = './datos/censo/'
RUTA_MGN_SHAPEFILES_DIR = './datos/mgn/conjunto_de_datos/'
RUTA_TIENDAS_INPUT_DIR = './datos/output/' # Donde está datos_train_con_objetivo.csv
RUTA_DATOS_OUTPUT_DIR = './datos/output/'   # Donde se guardará el resultado final

# Crear directorio de output si no existe
if not os.path.exists(RUTA_DATOS_OUTPUT_DIR):
    os.makedirs(RUTA_DATOS_OUTPUT_DIR)
    print(f"Directorio creado: {RUTA_DATOS_OUTPUT_DIR}")

# Archivos del Censo (asumiendo estos nombres, ajusta si es necesario)
ARCHIVO_CENSO_NL = RUTA_CENSO_CSV_DIR + 'nuevo_leon_censo_ageb.csv'
ARCHIVO_CENSO_TAM = RUTA_CENSO_CSV_DIR + 'tamaulipas_censo_ageb.csv'

# Shapefile de AGEBs y su columna clave (confirmado previamente)
SHAPEFILE_AGEB_NACIONAL = RUTA_MGN_SHAPEFILES_DIR + '00a.shp' 
COLUMNA_CVEGEO_EN_SHAPEFILE = 'CVEGEO' 
CRS_SHAPEFILE_AGEB = 'EPSG:6372'

# Archivo de tiendas con la variable objetivo
ARCHIVO_TIENDAS_CON_OBJETIVO = RUTA_TIENDAS_INPUT_DIR + 'datos_train_con_objetivo.csv'

# Archivo de salida
ARCHIVO_SALIDA_GPKG = RUTA_DATOS_OUTPUT_DIR + 'tiendas_train_final_con_censo_ageb.gpkg'

# --- Nombres de las columnas de claves geográficas COMO VIENEN DE SCITEL ---
COL_SCITEL_ENTIDAD = 'ENTIDAD'  
COL_SCITEL_MUNICIPIO = 'MUN'     
COL_SCITEL_LOCALIDAD = 'LOC'     
COL_SCITEL_AGEB = 'AGEB'        

COLUMNA_CVEGEO_CONSTRUIDA_EN_CENSO = 'CVEGEO_CONSTRUIDO' # Nombre que le daremos a la clave completa

print("--- Iniciando Proceso de Unión Geoespacial con Datos del Censo ---")

In [ ]:
# --- 1. Cargar y Concatenar Datos Censales de Nuevo León y Tamaulipas ---
try:
    print(f"Cargando datos del censo de Nuevo León desde: {ARCHIVO_CENSO_NL}")
    censo_nl_df = pd.read_csv(ARCHIVO_CENSO_NL)
    print(f"Cargando datos del censo de Tamaulipas desde: {ARCHIVO_CENSO_TAM}")
    censo_tam_df = pd.read_csv(ARCHIVO_CENSO_TAM)
    
    censo_ageb_df = pd.concat([censo_nl_df, censo_tam_df], ignore_index=True)
    print(f"\nDatos censales de NL y TAM cargados y concatenados: {len(censo_ageb_df)} registros AGEB.")
    print("Primeras filas de datos censales concatenados:")
    print(censo_ageb_df.head())
    print("\nColumnas en el DataFrame de censo concatenado:")
    print(censo_ageb_df.columns.tolist())

except FileNotFoundError as e:
    print(f"Error: No se encontró uno de los archivos del censo. Detalle: {e}")
    print(f"Verifica que '{ARCHIVO_CENSO_NL}' y '{ARCHIVO_CENSO_TAM}' existan.")
    raise
except Exception as e:
    print(f"Ocurrió un error al cargar o concatenar los archivos del censo: {e}")
    raise

In [ ]:
# --- 2. Construir la Columna CVEGEO en los Datos del Censo ---
# Verificar que las columnas necesarias para construir CVEGEO existan
columnas_clave_scitel = [COL_SCITEL_ENTIDAD, COL_SCITEL_MUNICIPIO, COL_SCITEL_LOCALIDAD, COL_SCITEL_AGEB]
missing_cols = [col for col in columnas_clave_scitel if col not in censo_ageb_df.columns]
if missing_cols:
    print(f"Error: Faltan las siguientes columnas clave en los datos del censo para construir CVEGEO: {missing_cols}")
    print(f"Columnas disponibles: {censo_ageb_df.columns.tolist()}")
    print("Por favor, ajusta las variables COL_SCITEL_ENTIDAD, etc., con los nombres correctos de tu descarga de SCITEL.")
    raise ValueError("Columnas para construir CVEGEO faltantes en datos del censo.")

try:
    print("\nConstruyendo CVEGEO a partir de las claves individuales del censo...")
    # Convertir a string y aplicar padding de ceros
    # Entidad: 2 dígitos
    censo_ageb_df[COL_SCITEL_ENTIDAD] = censo_ageb_df[COL_SCITEL_ENTIDAD].astype(str).str.zfill(2)
    # Municipio: 3 dígitos
    censo_ageb_df[COL_SCITEL_MUNICIPIO] = censo_ageb_df[COL_SCITEL_MUNICIPIO].astype(str).str.zfill(3)
    # Localidad: 4 dígitos
    censo_ageb_df[COL_SCITEL_LOCALIDAD] = censo_ageb_df[COL_SCITEL_LOCALIDAD].astype(str).str.zfill(4)
    # AGEB: 4 caracteres alfanuméricos (asegurar mayúsculas y padding si es necesario)
    # El padding de AGEB puede ser delicado si es alfanumérico y no solo numérico.
    # Si SCITEL ya lo da con 4 caracteres (ej. '001A'), solo se necesita .astype(str).str.upper()
    # Si puede ser numérico (ej. 123), entonces .str.pad(width=4, side='left', fillchar='0') es útil.
    # Como es alfanumérico, asumimos que ya tiene el largo correcto o que el padding de la izquierda con ceros no es el ideal
    # si hay letras. Generalmente INEGI ya lo da con el formato correcto '####' o '###L'.
    # Una conversión a string y a mayúsculas suele ser suficiente para la parte AGEB.
    censo_ageb_df[COL_SCITEL_AGEB] = censo_ageb_df[COL_SCITEL_AGEB].astype(str).str.upper().str.strip()
    # Validar longitud de la clave AGEB si se conoce (ej. 4 caracteres)
    # if not censo_ageb_df[COL_SCITEL_AGEB].str.len().eq(4).all():
    # print("Advertencia: No todas las claves de AGEB tienen 4 caracteres después de la conversión.")

    censo_ageb_df[COLUMNA_CVEGEO_CONSTRUIDA_EN_CENSO] = (
        censo_ageb_df[COL_SCITEL_ENTIDAD] +
        censo_ageb_df[COL_SCITEL_MUNICIPIO] +
        censo_ageb_df[COL_SCITEL_LOCALIDAD] +
        censo_ageb_df[COL_SCITEL_AGEB]
    )
    print(f"Columna '{COLUMNA_CVEGEO_CONSTRUIDA_EN_CENSO}' creada en el DataFrame del censo.")
    print(f"Ejemplo de CVEGEOs construidos: {censo_ageb_df[COLUMNA_CVEGEO_CONSTRUIDA_EN_CENSO].head().tolist()}")
except Exception as e:
    print(f"Error al construir CVEGEO en los datos del censo: {e}")
    raise

In [ ]:
# --- 3. Cargar el Shapefile Nacional de AGEBs ---
try:
    ageb_gdf_nacional = gpd.read_file(SHAPEFILE_AGEB_NACIONAL)
    print(f"\nShapefile nacional de AGEBs '{SHAPEFILE_AGEB_NACIONAL}' cargado: {len(ageb_gdf_nacional)} AGEBs.")
    print(f"CRS original del shapefile: {ageb_gdf_nacional.crs}")
    
    # Verificar y asignar CRS si es necesario (basado en exploración previa)
    if ageb_gdf_nacional.crs is None:
        print(f"Asignando CRS conocido al shapefile: {CRS_SHAPEFILE_AGEB}")
        ageb_gdf_nacional.set_crs(CRS_SHAPEFILE_AGEB, inplace=True)
    elif ageb_gdf_nacional.crs.to_string() != CRS_SHAPEFILE_AGEB:
        print(f"ADVERTENCIA: CRS del shapefile ({ageb_gdf_nacional.crs}) difiere del esperado ({CRS_SHAPEFILE_AGEB}). Revisar.")
        # Podrías optar por reproyectar si estás seguro de ambos CRS, o detener.
        # ageb_gdf_nacional = ageb_gdf_nacional.to_crs(CRS_SHAPEFILE_AGEB)

    if COLUMNA_CVEGEO_EN_SHAPEFILE not in ageb_gdf_nacional.columns:
        raise ValueError(f"Columna clave '{COLUMNA_CVEGEO_EN_SHAPEFILE}' no encontrada en shapefile. Columnas: {ageb_gdf_nacional.columns.tolist()}")
except Exception as e:
    print(f"Error al cargar el shapefile nacional de AGEBs: {e}")
    raise

In [ ]:
# --- 4. Preparar Claves para el Merge y Unir Censo a Shapefile de AGEBs ---
try:
    ageb_gdf_nacional[COLUMNA_CVEGEO_EN_SHAPEFILE] = ageb_gdf_nacional[COLUMNA_CVEGEO_EN_SHAPEFILE].astype(str).str.strip()
    # La columna COLUMNA_CVEGEO_CONSTRUIDA_EN_CENSO ya fue convertida a string y strip en el paso 2.
    
    print(f"\nRealizando merge entre AGEBs (shapefile) y Censo (CSV) usando claves: '{COLUMNA_CVEGEO_EN_SHAPEFILE}' (shapefile) y '{COLUMNA_CVEGEO_CONSTRUIDA_EN_CENSO}' (censo)")
    
    # Seleccionar solo las columnas necesarias del censo para evitar duplicados de claves
    columnas_censo_a_unir = [COLUMNA_CVEGEO_CONSTRUIDA_EN_CENSO] + \
                             [col for col in censo_ageb_df.columns if col not in columnas_clave_scitel and col != COLUMNA_CVEGEO_CONSTRUIDA_EN_CENSO]
    censo_para_merge_df = censo_ageb_df[columnas_censo_a_unir].drop_duplicates(subset=[COLUMNA_CVEGEO_CONSTRUIDA_EN_CENSO])


    ageb_con_censo_gdf = ageb_gdf_nacional.merge(
        censo_para_merge_df,
        left_on=COLUMNA_CVEGEO_EN_SHAPEFILE,
        right_on=COLUMNA_CVEGEO_CONSTRUIDA_EN_CENSO,
        how='left' 
    )
    print(f"Merge completado. GeoDataFrame 'ageb_con_censo_gdf' creado con {len(ageb_con_censo_gdf)} filas.")
    
    # Verificar cuántos AGEBs del shapefile encontraron correspondencia
    # Usaremos una columna que sabemos que viene del censo_df (que no sea la clave de unión)
    col_verificacion_censo = [col for col in columnas_censo_a_unir if col != COLUMNA_CVEGEO_CONSTRUIDA_EN_CENSO][0]
    agebs_sin_censo_match = ageb_con_censo_gdf[col_verificacion_censo].isnull().sum()
    print(f"Número de AGEBs del shapefile sin datos censales (de NL/TAM): {agebs_sin_censo_match} de {len(ageb_con_censo_gdf)} AGEBs totales.")
    # Es normal que muchas no tengan match si el shapefile es nacional y el censo es solo de NL y TAM.
    
    # Si la columna clave del censo se añadió y es diferente de la del shapefile, podemos eliminarla
    if COLUMNA_CVEGEO_CONSTRUIDA_EN_CENSO != COLUMNA_CVEGEO_EN_SHAPEFILE and COLUMNA_CVEGEO_CONSTRUIDA_EN_CENSO in ageb_con_censo_gdf.columns:
        ageb_con_censo_gdf = ageb_con_censo_gdf.drop(columns=[COLUMNA_CVEGEO_CONSTRUIDA_EN_CENSO])
        print(f"Columna redundante '{COLUMNA_CVEGEO_CONSTRUIDA_EN_CENSO}' eliminada.")

except Exception as e:
    print(f"Error durante el merge de AGEBs y Censo: {e}")
    raise

In [ ]:
# --- 5. Cargar Datos de Tiendas y Convertir a GeoDataFrame ---
try:
    tiendas_df = pd.read_csv(ARCHIVO_TIENDAS_CON_OBJETIVO)
    print(f"\nDatos de tiendas desde '{ARCHIVO_TIENDAS_CON_OBJETIVO}' cargados: {len(tiendas_df)} tiendas.")
    
    tiendas_gdf = gpd.GeoDataFrame(
        tiendas_df,
        geometry=gpd.points_from_xy(tiendas_df.LONGITUD_NUM, tiendas_df.LATITUD_NUM),
        crs="EPSG:4326"  # WGS84 para lat/lon
    )
    print(f"GeoDataFrame de tiendas creado. CRS inicial: {tiendas_gdf.crs}")
except FileNotFoundError:
    print(f"Error: No se encontró '{ARCHIVO_TIENDAS_CON_OBJETIVO}'. Ejecuta el notebook 02_... primero.")
    raise
except Exception as e:
    print(f"Error al cargar tiendas o crear GeoDataFrame de tiendas: {e}")
    raise

In [ ]:
# --- 6. Asegurar que Tiendas y AGEBs+Censo tengan el mismo CRS ---
if ageb_con_censo_gdf.crs is None:
    raise ValueError("CRS de 'ageb_con_censo_gdf' es None. No se puede proceder.")

if tiendas_gdf.crs != ageb_con_censo_gdf.crs:
    print(f"\nTransformando CRS de tiendas ({tiendas_gdf.crs}) para que coincida con CRS de AGEBs ({ageb_con_censo_gdf.crs})...")
    try:
        tiendas_gdf_proyectado = tiendas_gdf.to_crs(ageb_con_censo_gdf.crs)
        print(f"Nuevo CRS de tiendas: {tiendas_gdf_proyectado.crs}")
    except Exception as e:
        print(f"Error durante la transformación de CRS de tiendas: {e}")
        raise
else:
    print("\nCRS de tiendas y AGEBs ya coinciden.")
    tiendas_gdf_proyectado = tiendas_gdf


In [ ]:
# --- 7. Realizar la Unión Espacial (sjoin) ---
print("\nRealizando unión espacial (sjoin) entre tiendas y AGEBs con censo...")
try:
    # Seleccionar columnas del GeoDataFrame de AGEB+Censo para la unión
    # Esto es para evitar añadir TODAS las columnas del shapefile original si no son necesarias
    # y solo las columnas del censo + la clave CVEGEO + la geometría.
    columnas_del_censo_en_ageb_gdf = [col for col in censo_para_merge_df.columns if col != COLUMNA_CVEGEO_CONSTRUIDA_EN_CENSO]
    columnas_para_sjoin_desde_ageb = [COLUMNA_CVEGEO_EN_SHAPEFILE] + columnas_del_censo_en_ageb_gdf + ['geometry']
    # Filtrar solo columnas que realmente existen
    columnas_para_sjoin_desde_ageb_existentes = [c for c in columnas_para_sjoin_desde_ageb if c in ageb_con_censo_gdf.columns]
    columnas_para_sjoin_desde_ageb_existentes = list(dict.fromkeys(columnas_para_sjoin_desde_ageb_existentes))


    ageb_para_sjoin_final_gdf = ageb_con_censo_gdf[columnas_para_sjoin_desde_ageb_existentes]

    tiendas_final_gdf = gpd.sjoin(
        tiendas_gdf_proyectado,
        ageb_para_sjoin_final_gdf, 
        how='left',
        predicate='within' 
    )
    print(f"Unión espacial completada. Dataset final: {len(tiendas_final_gdf)} filas.")
    if 'index_right' in tiendas_final_gdf.columns:
        tiendas_final_gdf = tiendas_final_gdf.drop(columns=['index_right'])
        print("Columna 'index_right' eliminada.")
except Exception as e:
    print(f"Error durante la unión espacial (sjoin): {e}")
    raise

In [ ]:
# --- 8. Verificar y Guardar el Resultado Final ---
print("\n--- Verificación del Resultado Final ---")
tiendas_sin_ageb_data = tiendas_final_gdf[COLUMNA_CVEGEO_EN_SHAPEFILE].isnull().sum()
print(f"Número de tiendas que NO cayeron dentro de un AGEB (o AGEB sin datos censales): {tiendas_sin_ageb_data} de {len(tiendas_final_gdf)}")

print("\nPrimeras filas del dataset enriquecido final:")
pd.set_option('display.max_columns', None)
print(tiendas_final_gdf.head())

print("\nColumnas del dataset enriquecido final:")
print(tiendas_final_gdf.columns.tolist())

try:
    tiendas_final_gdf.to_file(ARCHIVO_SALIDA_GPKG, driver='GPKG', layer='tiendas_censo_ageb_nl_tam')
    print(f"\nDataset enriquecido guardado como GeoPackage en '{ARCHIVO_SALIDA_GPKG}'")
except Exception as e:
    print(f"Error al guardar el archivo de salida GeoPackage: {e}")

print("\n--- Proceso de Unión Geoespacial con Datos del Censo (AGEB) Completado ---")